# Aula 5 - Satisfação de restrições

Neste notebook, formularemos o problema das n-rainhas como um CSP e implementaremos o algoritmo Backtracking-Search com diferentes heurísticas para resolver o mesmo. O objetivo aqui é entender como este algoritmo funciona. Ao final, você deverá realizar os exercícios propostos.

Primeiramente, vamos criar algumas funções e classes de apoio.

In [ ]:
import random
import numpy as np
from collections import defaultdict

def first(iterable, default=None):
    """Return the first element of an iterable; or default."""
    return next(iter(iterable), default)

def count(seq):
    """Count the number of items in sequence that are interpreted as true."""
    return sum(map(bool, seq))

class UniversalDict:
    """A universal dict maps any key to the same value. We use it here
    as the domains dict for CSPs in which all variables have the same domain.
    >>> d = UniversalDict(42)
    >>> d['life']
    42
    """

    def __init__(self, value): 
      self.value = value

    def __getitem__(self, key): 
      return self.value

    def __repr__(self): 
      return '{{Any: {0!r}}}'.format(self.value)

class Problem:
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

Agora, vamos criar uma classe que formalize um problema de satisfação de restrições (CSP).

In [ ]:
class CSP(Problem):
    """This class describes finite-domain Constraint Satisfaction Problems.
    A CSP is specified by the following inputs:
        variables   A list of variables; each is atomic (e.g. int or string).
        domains     A dict of {var:[possible_value, ...]} entries.
        neighbors   A dict of {var:[var,...]} that for each variable lists
                    the other variables that participate in constraints.
        constraints A function f(A, a, B, b) that returns true if neighbors
                    A, B satisfy the constraint when they have values A=a, B=b

    In the textbook and in most mathematical definitions, the
    constraints are specified as explicit pairs of allowable values,
    but the formulation here is easier to express and more compact for
    most cases. (For example, the n-Queens problem can be represented
    in O(n) space using this notation, instead of O(N^4) for the
    explicit representation.) In terms of describing the CSP as a
    problem, that's all there is.

    However, the class also supports data structures and methods that help you
    solve CSPs by calling a search function on the CSP. Methods and slots are
    as follows, where the argument 'a' represents an assignment, which is a
    dict of {var:val} entries:
        assign(var, val, a)     Assign a[var] = val; do other bookkeeping
        unassign(var, a)        Do del a[var], plus other bookkeeping
        nconflicts(var, val, a) Return the number of other variables that
                                conflict with var=val
        curr_domains[var]       Slot: remaining consistent values for var
                                Used by constraint propagation routines.
    The following methods are used only by graph_search and tree_search:
        actions(state)          Return a list of actions
        result(state, action)   Return a successor of state
        goal_test(state)        Return true if all constraints satisfied
    The following are just for debugging purposes:
        nassigns                Slot: tracks the number of assignments made
        display(a)              Print a human-readable representation
    """

    def __init__(self, variables, domains, neighbors, constraints):
        """Construct a CSP problem. If variables is empty, it becomes domains.keys()."""
        variables = variables or list(domains.keys())
        self.variables = variables
        self.domains = domains
        self.neighbors = neighbors
        self.constraints = constraints
        self.initial = ()
        self.curr_domains = None
        self.nassigns = 0

    def assign(self, var, val, assignment):
        """Add {var: val} to assignment; Discard the old value if any."""
        assignment[var] = val
        self.nassigns += 1

    def unassign(self, var, assignment):
        """Remove {var: val} from assignment.
        DO NOT call this if you are changing a variable to a new value;
        just call assign for that."""
        if var in assignment:
            del assignment[var]

    def nconflicts(self, var, val, assignment):
        """Return the number of conflicts var=val has with other variables."""

        # Subclasses may implement this more efficiently
        def conflict(var2):
            return (var2 in assignment and
                    not self.constraints(var, val, var2, assignment[var2]))

        return count(conflict(v) for v in self.neighbors[var])

    def display(self, assignment):
        """Show a human-readable representation of the CSP."""
        # Subclasses can print in a prettier way, or display with a GUI
        print('CSP:', self, 'with assignment:', assignment)

    # These methods are for the tree and graph-search interface:

    def actions(self, state):
        """Return a list of applicable actions: nonconflicting
        assignments to an unassigned variable."""
        if len(state) == len(self.variables):
            return []
        else:
            assignment = dict(state)
            var = first([v for v in self.variables if v not in assignment])
            return [(var, val) for val in self.domains[var]
                    if self.nconflicts(var, val, assignment) == 0]

    def result(self, state, action):
        """Perform an action and return the new state."""
        (var, val) = action
        return state + ((var, val),)

    def goal_test(self, state):
        """The goal is to assign all variables, with all constraints satisfied."""
        assignment = dict(state)
        return (len(assignment) == len(self.variables)
                and all(self.nconflicts(variables, assignment[variables], assignment) == 0
                        for variables in self.variables))

    # These are for constraint propagation

    def support_pruning(self):
        """Make sure we can prune values from domains. (We want to pay
        for this only if we use it.)"""
        if self.curr_domains is None:
            self.curr_domains = {v: list(self.domains[v]) for v in self.variables}

    def suppose(self, var, value):
        """Start accumulating inferences from assuming var=value."""
        self.support_pruning()
        removals = [(var, a) for a in self.curr_domains[var] if a != value]
        self.curr_domains[var] = [value]
        return removals

    def prune(self, var, value, removals):
        """Rule out var=value."""
        self.curr_domains[var].remove(value)
        if removals is not None:
            removals.append((var, value))

    def choices(self, var):
        """Return all values for var that aren't currently ruled out."""
        return (self.curr_domains or self.domains)[var]

    def infer_assignment(self):
        """Return the partial assignment implied by the current inferences."""
        self.support_pruning()
        return {v: self.curr_domains[v][0]
                for v in self.variables if 1 == len(self.curr_domains[v])}

    def restore(self, removals):
        """Undo a supposition and all inferences from it."""
        for B, b in removals:
            self.curr_domains[B].append(b)

Agora podemos criar o algoritmo Backtracking-Search. Este algoritmo é implementado pela função `backtracking_search`, que recebe um CSP (`csp`), além de funções para seleção de variáveis (`select_unassigned_variable`), seleção de domínios de variáveis (`order_domain_values`) e inferência (`inference`). Por padrão, nossa implementação recebe as funções `first_unassigned_variable` (que retorna a primeira variável livre), `unordered_domain_values` (que retorna todos os valores disponíveis) e `no_inference` (que não faz qualquer inferência). Ao final, nossa função `backtracking_search` retorna a alocação encontrada (ou `None`, caso nenhuma alocação válida tenha sido encontrada).


In [ ]:
def first_unassigned_variable(assignment, csp):
    return first([var for var in csp.variables if var not in assignment])

def unordered_domain_values(var, assignment, csp):
    return csp.choices(var)

def no_inference(csp, var, value, assignment, removals):
    return True

def backtracking_search(csp, select_unassigned_variable=first_unassigned_variable,
                        order_domain_values=unordered_domain_values, inference=no_inference):

    def backtrack(assignment):
        if len(assignment) == len(csp.variables):
            return assignment
        var = select_unassigned_variable(assignment, csp)
        for value in order_domain_values(var, assignment, csp):
            if 0 == csp.nconflicts(var, value, assignment):
                csp.assign(var, value, assignment)
                removals = csp.suppose(var, value)
                if inference(csp, var, value, assignment, removals):
                    result = backtrack(assignment)
                    if result is not None:
                        return result
                csp.restore(removals)
        csp.unassign(var, assignment)
        return None

    result = backtrack({})
    assert result is None or csp.goal_test(result)
    return result

Note que o algoritmo acima não é muito eficiente na seleção de variáveis/valores e no uso de inferência. Vamos resolver isso com técnica mais sofisticadas. 

Primeiramente, abaixo empregamos a heurística "minimum remaining values" (`mrv`), que recebe um CSP (`csp`) e retorna a variável com menos possibilidades de valores disponíveis.

In [ ]:
def mrv(assignment, csp):
    return argmin_random_tie([v for v in csp.variables if v not in assignment],
                             key=lambda var: num_legal_values(csp, var, assignment))

def num_legal_values(csp, var, assignment):
    if csp.curr_domains:
        return len(csp.curr_domains[var])
    else:
        return count(csp.nconflicts(var, val, assignment) == 0 for val in csp.domains[var])

def argmin_random_tie(seq, key=lambda x: x):
    """Return a minimum element of seq; break ties at random."""
    items = list(seq)
    random.shuffle(items)
    return min(items, key=key)

Em termos de seleção de valores, abaixo apresentamos a heurística "least constrained value" (`lcv`), que recebe um CSP (`csp`), uma variável (`var`) e uma alocação parcial (`assignment`) e retorna o valor de `var` que restringe menos as demais variáveis.

In [ ]:
def lcv(var, assignment, csp):
    return sorted(csp.choices(var), key=lambda val: csp.nconflicts(var, val, assignment))

Finalmente, abaixo apresentamos o mecanismo `forward_checking` para inferências. Este mecanismo recebe um CSP (`csp`), uma variável (`var`), um valor (`value`), uma alocação parcial (`assignment`) e uma lista de remoções (`removals`). O algoritmo então antecipa possíveis falhas e poda os valores correspondentes. 

In [ ]:
def forward_checking(csp, var, value, assignment, removals):
    csp.support_pruning()
    for B in csp.neighbors[var]:
        if B not in assignment:
            for b in csp.curr_domains[B][:]:
                if not csp.constraints(var, value, B, b):
                    csp.prune(B, b, removals)
            if not csp.curr_domains[B]:
                return False
    return True

Definidos o algoritmo de backtracking e a formulação geral de CSPs, agora podemos criar uma classe para o problema N-Queens. Faremos isto através da classe `NQueensCSP`, subclasse de `CSP`, que consegue representar instâncias arbitrárias do problema (como a instância clássica 8-rainhas).

In [ ]:
class NQueensCSP(CSP):
    """Make a CSP for the nQueens problem for search with min_conflicts.
    Suitable for large n, it uses only data structures of size O(n).
    Think of placing queens one per column, from left to right.
    That means position (x, y) represents (var, val) in the CSP.
    The main structures are three arrays to count queens that could conflict:
        rows[i]      Number of queens in the ith row (i.e val == i)
        downs[i]     Number of queens in the \ diagonal
                     such that their (x, y) coordinates sum to i
        ups[i]       Number of queens in the / diagonal
                     such that their (x, y) coordinates have x-y+n-1 = i
    We increment/decrement these counts each time a queen is placed/moved from
    a row/diagonal. So moving is O(1), as is nconflicts.  But choosing
    a variable, and a best value for the variable, are each O(n).
    If you want, you can keep track of conflicted variables, then variable
    selection will also be O(1).
    >>> len(backtracking_search(NQueensCSP(8)))
    8
    """

    def __init__(self, n):
        """Initialize data structures for n Queens."""
        CSP.__init__(self, list(range(n)), UniversalDict(list(range(n))),
                     UniversalDict(list(range(n))), self.queen_constraint)

        self.rows = [0] * n
        self.ups = [0] * (2 * n - 1)
        self.downs = [0] * (2 * n - 1)

    def nconflicts(self, var, val, assignment):
        """The number of conflicts, as recorded with each assignment.
        Count conflicts in row and in up, down diagonals. If there
        is a queen there, it can't conflict with itself, so subtract 3."""
        n = len(self.variables)
        c = self.rows[val] + self.downs[var + val] + self.ups[var - val + n - 1]
        if assignment.get(var, None) == val:
            c -= 3
        return c

    def assign(self, var, val, assignment):
        """Assign var, and keep track of conflicts."""
        oldval = assignment.get(var, None)
        if val != oldval:
            if oldval is not None:  # Remove old val if there was one
                self.record_conflict(assignment, var, oldval, -1)
            self.record_conflict(assignment, var, val, +1)
            CSP.assign(self, var, val, assignment)

    def unassign(self, var, assignment):
        """Remove var from assignment (if it is there) and track conflicts."""
        if var in assignment:
            self.record_conflict(assignment, var, assignment[var], -1)
        CSP.unassign(self, var, assignment)

    def record_conflict(self, assignment, var, val, delta):
        """Record conflicts caused by addition or deletion of a Queen."""
        n = len(self.variables)
        self.rows[val] += delta
        self.downs[var + val] += delta
        self.ups[var - val + n - 1] += delta

    def display(self, assignment):
        """Print the queens and the nconflicts values (for debugging)."""
        n = len(self.variables)
        for val in range(n):
            for var in range(n):
                if assignment.get(var, '') == val:
                    ch = 'Q'
                elif (var + val) % 2 == 0:
                    ch = '.'
                else:
                    ch = '-'
                print(ch, end=' ')
            print('    ', end=' ')
            for var in range(n):
                if assignment.get(var, '') == val:
                    ch = '*'
                else:
                    ch = ' '
                print(str(self.nconflicts(var, val, assignment)) + ch, end=' ')
            print()

    def queen_constraint(self, A, a, B, b):
      """Constraint is satisfied (true) if A, B are really the same variable,
      or if they are not in the same row, down diagonal, or up diagonal."""
      return A == B or (a != b and A + a != B + b and A - a != B - b)

De modo a facilitar a visualização das soluções, abaixo criamos ainda uma função de apoio. A função `plot_NQueens` recebe uma solução (`solution`) e a imprime de modo organizado.

In [ ]:
def plot_NQueens(solution):
    n = len(solution)
    line = '-'*(n*4+1)
    print(line)
    for c in range(n):
      string = '|'
      for r in range(n):
        p = ' '
        if c == solution[r]:
          p = 'X'
        string += ' ' + p + ' |'
      print(string)
      print('-'*(n*4+1))

Agora podemos finalmente testar nosso algoritmo de backtracking no problema N-Queens. Abaixo, criamos a instância 8-rainhas. Em seguida, rodamos o algoritmo `backtracking_search` nesta instância, utilizando as 
heurísticas de seleção de variáveis/valores e de inferência criadas por último.

In [ ]:
eight_queens = NQueensCSP(8)
solution = backtracking_search(
    eight_queens, 
    select_unassigned_variable=mrv,
    order_domain_values=lcv,
    inference=no_inference
    )
plot_NQueens(solution)
print(solution)

Como pode ser observado, o resultado encontrado é uma solução válida para o problema. 

Lembre-se que criamos funções simples e mais sofisticadas para a seleção de variáveis/valores e de inferência. Para poder comparar ambas, vamos testar uma instância maior do problema, o 20-rainhas. 

In [ ]:
import time

twenty_queens1 = NQueensCSP(20)
twenty_queens2 = NQueensCSP(20)

start = time.time()
solution1 = backtracking_search(
    twenty_queens1, 
    select_unassigned_variable=first_unassigned_variable,
    order_domain_values=unordered_domain_values, 
    inference=no_inference
    )
runtime1 = time.time() - start
plot_NQueens(solution1)

start = time.time()
solution2 = backtracking_search(
    twenty_queens2, 
    select_unassigned_variable=mrv,
    order_domain_values=lcv,
    #inference=no_inference
    inference=forward_checking
    )
runtime2 = time.time() - start
plot_NQueens(solution2)

print('Runtime 1: %.2fs' % (runtime1))
print('Runtime 2: %.2fs' % (runtime2))

Conforme pode ser observado acima, as heurísticas e inferências realizadas pelo segundo algoritmo o tornam consideravelmente mais rápido que a versão mais trivial do mesmo.

### Exercício 1

Modele o problema Sudoku como um CSP. Para isto, crie a classe `SudokuCSP`. Ao ser instanciada, esta classe deve receber como argumento uma instância do problema como o exemplo (a) da figura abaixo (vide Exercício 5 para representação desta instância). Ainda na inicialização, é preciso converter a instância recebida em um grafo seguindo a formalização de um CSP (vide classe correspondente). Dica: para denotar posições fixas (ex: no exemplo, a posição A3 tem valor 3 fixo), crie domínios com um valor único para estas variáveis.

![alt text](https://i.imgur.com/YRbZf7l.png)

In [ ]:
class SudokuCSP(CSP):
    
    def __init__(self, instance):

        self.rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
        self.columns = list(range(1,10))

        # create the main elements of the CSP
        variables = []
        domains = {}
        neighbors = {}

        # populate the main elements of the CSP
        for row in self.rows:
            for col in self.columns:
                var = '%s%d' % (row, col)
                variables.append(var)
                domains[var] = list(range(1,10))
                neighbors[var] = self.list_neighbors_var(row, col)

        # process the variables already assigned in the instance
        for rowid, row in enumerate(self.rows):
            for colid, col in enumerate(self.columns):
                if instance[rowid][colid] != 0:
                    var = '%s%d' % (row, col)
                    domains[var] = [instance[rowid][colid]]
        
        # initialise the CSP
        CSP.__init__(self, 
                     variables, #list(range(n)), 
                     domains, #UniversalDict(list(range(n))),
                     neighbors, #UniversalDict(list(range(n))), 
                     self.sudoku_constraints)

    def list_neighbors_var(self, row, col):

        neighbors = []

        # same column
        for r in self.rows:
            neigh = '%s%d' % (r, col)
            if neigh not in neighbors:
                neighbors.append(neigh)

        # same row
        for c in self.columns:
            neigh = '%s%d' % (row, c)
            if neigh not in neighbors:
                neighbors.append(neigh)

        # same sub-region
        rowid = self.rows.index(row)
        reg_cs = list(range(int((col-1) / 3) * 3 + 1, int((col-1) / 3) * 3 + 3 + 1)) # columns in sub-region
        reg_rs = self.rows[int(rowid / 3) * 3 : int(rowid / 3) * 3 + 3] # rows in sub-region
        for r in reg_rs:
            for c in reg_cs:
                neigh = '%s%d' % (r, c)
                if neigh not in neighbors:
                    neighbors.append(neigh)

        return neighbors

    def sudoku_constraints(self, A, a, B, b):
        return A == B or (A != B and a != b)

    def display(self, assignment):
        rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
        columns = list(range(1,10))
        print('-'*37)
        for r in rows:
            line = '|'
            for c in columns:
                neigh = '%s%d' % (r, c)
                line += ' %d |' % (assignment[neigh])
            print(line)
            print('-'*37)

### Exercício 2

Resolva a instância de Sudoku apresentada na figura (a) do exercício 4 utilizando o algoritmo `backtracking_search`. Compare a execução do algoritmo com e sem as heurísticas de seleção de variáveis/valores e de inferência.

In [ ]:
instance = [
  [0, 0, 3, 0, 2, 0, 6, 0, 0],
  [9, 0, 0, 3, 0, 5, 0, 0, 1],
  [0, 0, 1, 8, 0, 6, 4, 0, 0],
  [0, 0, 8, 1, 0, 2, 9, 0, 0],
  [7, 0, 0, 0, 0, 0, 0, 0, 8],
  [0, 0, 6, 7, 0, 8, 2, 0, 0],
  [0, 0, 2, 6, 0, 9, 5, 0, 0],
  [8, 0, 0, 2, 0, 3, 0, 0, 9],
  [0, 0, 5, 0, 1, 0, 3, 0, 0]
]

sudoku = SudokuCSP(instance)
solution = backtracking_search(
    sudoku, 
    select_unassigned_variable=mrv,
    order_domain_values=lcv,
    inference=forward_checking
    )
sudoku.display(solution)

In [ ]:
# no heuristics/inference
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku
    )
runtime1 = time.time() - start
sudoku.display(solution)

# with variable selection heuristics
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku, 
    select_unassigned_variable=mrv
    )
runtime2 = time.time() - start
sudoku.display(solution)

# with value selection heuristics
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku, 
    order_domain_values=lcv
    )
runtime3 = time.time() - start
sudoku.display(solution)

# with variable and value selection heuristics
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku, 
    select_unassigned_variable=mrv,
    order_domain_values=lcv
    )
runtime4 = time.time() - start
sudoku.display(solution)

# with inference
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku, 
    inference=forward_checking
    )
runtime5 = time.time() - start
sudoku.display(solution)

# with heuristics and inference
sudoku = SudokuCSP(instance)
start = time.time()
solution = backtracking_search(
    sudoku, 
    select_unassigned_variable=mrv,
    order_domain_values=lcv,
    inference=forward_checking
    )
runtime6 = time.time() - start
sudoku.display(solution)

print('Runtime 1: %.2fs' % (runtime1))
print('Runtime 2: %.2fs' % (runtime2))
print('Runtime 3: %.2fs' % (runtime3))
print('Runtime 4: %.2fs' % (runtime4))
print('Runtime 5: %.2fs' % (runtime5))
print('Runtime 6: %.2fs' % (runtime6))

(fim)